In [ ]:
#r "nuget: Microsoft.DotNet.Interactive.SqlServer, *-*"

In [ ]:
#!connect mssql --kernel-name sql2025 --connection-string "Server=SQL2025;TrustServerCertificate=True;Integrated Security=True"

In [4]:
SELECT @@VERSION

(1 row affected)

(No column name)
Microsoft SQL Server 2025 (RC0) - 17.0.900.3 (X64) Aug 8 2025 18:30:17 Copyright (C) 2025 Microsoft Corporation Enterprise Evaluation Edition (64-bit) on Windows Server 2022 Standard 10.0 <X64> (Build 20348: ) (Hypervisor)


In [5]:
USE LinkedInCourseDB

Commands completed successfully.

In [6]:
ALTER TABLE SQLBooks
ADD CONSTRAINT PK_a_Id PRIMARY KEY (Id)

Commands completed successfully.

In [7]:
CREATE VECTOR INDEX vec_idx ON SQLBooks([embeddings])
WITH (
    metric = 'cosine',
    type = 'diskann',
    maxdop = 8
)

Commands completed successfully.

In [8]:
UPDATE SQLBooks set title = title + '!'

Error: Msg 42231, Level 16, State 1, Line 1
Data modification statement failed because table 'SQLBooks' has a vector index on it.

In [9]:
SELECT Title FROM SQLBooks where title like '%datawarehouse%'

(0 rows affected)

Info: No rows were returned for query 0 in batch 0.

In [10]:
DECLARE @search_text NVARCHAR(MAX) = 'datawarehouse'
DECLARE @search_vector VECTOR(768) = AI_GENERATE_EMBEDDINGS(@search_text USE MODEL ollama)

SELECT 
    MainAuthor,title,
    s.distance
FROM vector_search(
    table = SQLBooks AS t,
    column = [embeddings],
    similar_to = @search_vector,
    metric = 'cosine',
    top_n = 3
) AS s
ORDER BY s.distance

(3 rows affected)

MainAuthor,title,distance
Vincent Rainardi,Building a Data Warehouse,"0,29426318407058716"
Michael J. Corey,SQL server 7 data warehousing,"0,3347949981689453"
Joy Mundy,The Microsoft data warehouse toolkit,"0,35621267557144165"


In [11]:
SET STATISTICS IO ON

Commands completed successfully.

In [12]:
DECLARE @search_text NVARCHAR(MAX) = 'datawarehouse'
DECLARE @search_vector VECTOR(768) = AI_GENERATE_EMBEDDINGS(@search_text USE MODEL ollama)

SELECT 
    MainAuthor,title,
    s.distance
FROM vector_search(
    table = SQLBooks AS t,
    column = [embeddings],
    similar_to = @search_vector,
    metric = 'cosine',
    top_n = 3
) AS s
ORDER BY s.distance

(3 rows affected)

Table 'SQLBooks'. Scan count 0, logical reads 723, physical reads 0, page server reads 0, read-ahead reads 0, page server read-ahead reads 0, lob logical reads 0, lob physical reads 0, lob page server reads 0, lob read-ahead reads 0, lob page server read-ahead reads 0.

Table 'Worktable'. Scan count 11, logical reads 187, physical reads 0, page server reads 0, read-ahead reads 0, page server read-ahead reads 0, lob logical reads 0, lob physical reads 0, lob page server reads 0, lob read-ahead reads 0, lob page server read-ahead reads 0.

Table 'vector_index_Graph_Edge_table_1221579390_1152000'. Scan count 0, logical reads 24, physical reads 0, page server reads 0, read-ahead reads 0, page server read-ahead reads 0, lob logical reads 0, lob physical reads 0, lob page server reads 0, lob read-ahead reads 0, lob page server read-ahead reads 0.

MainAuthor,title,distance
Vincent Rainardi,Building a Data Warehouse,"0,29426318407058716"
Michael J. Corey,SQL server 7 data warehousing,"0,3347949981689453"
Joy Mundy,The Microsoft data warehouse toolkit,"0,35621267557144165"


In [13]:
DECLARE @search_text NVARCHAR(MAX) = 'datawarehouse'
DECLARE @search_vector VECTOR(768) = AI_GENERATE_EMBEDDINGS(@search_text USE MODEL ollama)
 
SELECT Top 3 MainAuthor,title,
    vector_distance('cosine', @search_vector, embeddings) AS distance
FROM SQLBooks  
WHERE MainAuthor <> ''
ORDER BY distance

(3 rows affected)

Table 'SQLBooks'. Scan count 1, logical reads 1718, physical reads 0, page server reads 0, read-ahead reads 0, page server read-ahead reads 0, lob logical reads 0, lob physical reads 0, lob page server reads 0, lob read-ahead reads 0, lob page server read-ahead reads 0.

MainAuthor,title,distance
Vincent Rainardi,Building a Data Warehouse,"0,29426318407058716"
Michael J. Corey,SQL server 7 data warehousing,"0,3347949981689453"
Joy Mundy,The Microsoft data warehouse toolkit,"0,35621267557144165"


In [14]:
#!connect mssql --kernel-name sql2025-stackoverflow --connection-string "Server=sqlstackoverflow\ctp21;TrustServerCertificate=True;Integrated Security=True"

Kernel added: #!sql-sql2025-stackoverflow

In [15]:
use StackOverflowExcerpt

Commands completed successfully.

In [16]:
SELECT SUM(row_count) AS approx_rows FROM sys.dm_db_partition_stats WHERE object_id = OBJECT_ID('dbo.Posts_Excerpt_large')		
 

(1 row affected)

approx_rows
10000000


In [17]:
SET STATISTICS IO ON

Commands completed successfully.

In [18]:
DECLARE @search_text NVARCHAR(MAX) = 'i want to build a website using visual basic and active server pages'
DECLARE @search_vector VECTOR(768) = AI_GENERATE_EMBEDDINGS(@search_text USE MODEL ollama)
 
SELECT top 2
    id,title,
    vector_distance('cosine', @search_vector, p.embeddings) AS distance
FROM Posts_Excerpt_large p
 
ORDER BY distance

(2 rows affected)

Table 'Posts_Excerpt_Large'. Scan count 15, logical reads 8569367, physical reads 2420, page server reads 0, read-ahead reads 8546682, page server read-ahead reads 0, lob logical reads 3411, lob physical reads 87, lob page server reads 0, lob read-ahead reads 0, lob page server read-ahead reads 0.

id,title,distance
1778032,How to design and develop a web page in asp.net using vb.net?,"0,25530630350112915"
3059786,how to learn web service using vb.net,"0,2572418451309204"


In [19]:
DECLARE @search_text NVARCHAR(MAX) = 'i want to build a website using visual basic and active server pages'
DECLARE @search_vector VECTOR(768) = AI_GENERATE_EMBEDDINGS(@search_text USE MODEL ollama)

SELECT
    t.id,title,    s.distance
FROM vector_search(
    table = Posts_Excerpt_large AS t,
    column = [embeddings],
    similar_to = @search_vector,
    metric = 'cosine',
    top_n = 2
) AS s  
ORDER BY s.distance

(2 rows affected)

Table 'Posts_Excerpt_Large'. Scan count 0, logical reads 5864, physical reads 1250, page server reads 0, read-ahead reads 0, page server read-ahead reads 0, lob logical reads 1, lob physical reads 1, lob page server reads 0, lob read-ahead reads 0, lob page server read-ahead reads 0.

Table 'Worktable'. Scan count 23, logical reads 443, physical reads 0, page server reads 0, read-ahead reads 0, page server read-ahead reads 0, lob logical reads 0, lob physical reads 0, lob page server reads 0, lob read-ahead reads 0, lob page server read-ahead reads 0.

Table 'vector_index_Graph_Edge_table_46623209_1152000'. Scan count 0, logical reads 112, physical reads 59, page server reads 0, read-ahead reads 0, page server read-ahead reads 0, lob logical reads 0, lob physical reads 0, lob page server reads 0, lob read-ahead reads 0, lob page server read-ahead reads 0.

id,title,distance
1778032,How to design and develop a web page in asp.net using vb.net?,"0,25530630350112915"
3059786,how to learn web service using vb.net,"0,2572418451309204"
